In [1]:
import os
from pathlib import Path

In [4]:
%pwd

'c:\\Users\\Hp\\Videos\\classification implementation - machine learning with MLFlow'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Hp\\Videos\\classification implementation - machine learning with MLFlow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    learning_rate: float
    n_estimators: int
    max_depth: int
    subsample: float
    colsample_bytree: float
    gamma: float
    reg_alpha: float
    reg_lambda: float
    min_child_weight: int
    eval_metric: str
    early_stopping_rounds: int
    tree_method: str
    target_column: str

In [6]:
## create configuration manager
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer,
        params = self.params.XGBClassifier,
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_daa_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            learning_rate=params.learning_rate,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            subsample=params.subsample,
            colsample_bytree=params.colsample_bytree,
            gamma=params.gamma,
            reg_alpha=params.eg_alpha,
            reg_lambda=params.reg_lambda,
            min_child_weight=params.min_child_weight,
            eval_metric=params.eval_metric,
            early_stopping_rounds=params.early_stopping_rounds,
            tree_method=params.tree_method,
            target_column=schema.target_column
        )

        return model_trainer_config

In [18]:
import pandas as pd
import os
from mlProject import logger
from xgboost import XGBClassifier
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from mlProject.utils.common import feature_processing

ImportError: cannot import name 'feature_processing' from 'mlProject.utils.common' (c:\users\hp\videos\classification implementation - machine learning with mlflow\src\mlProject\utils\common.py)

In [ ]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data([self.config.target_column])
        test_y = test_data([self.config.target_column])

        